In [16]:
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
from exp.exp_imputation import Exp_Imputation
from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
from exp.exp_anomaly_detection import Exp_Anomaly_Detection
from exp.exp_classification import Exp_Classification
from data_provider.data_creat import *
import akshare as ak
import datetime
from torch.utils.tensorboard import SummaryWriter

In [17]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    is_training = 1
    model_id = 'Stock_96_96'
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'
    
    '''股票数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20151201'  # 20151201 下载数据的开始日期,0就是公司上市时间
    end_date = '-1'  # 下载数据的结束日期,如果0则到最后一天,如果-1是昨天.
    final_data_feat =  ['index', 'Volume','Tom_Chg'] # 删除不需要列的标签
    label_n = 5 # 预测未来连续多少天的收益率
    # 标签是否替换成1或者0
    zhangfu = 0.10  # 预测涨幅大于等于3%的为1，小于3%的为0
    label_ch = True  # 如果是True ，预测n天以后上涨大于变量zhangfu为1，小于为0
    
    
    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    root_path = './dataset/Stock/'
    data_path = 'Stock.csv'
    # 数据拼接不同股票中间添加间隔的数量
    data_addzero = 60
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'MS'
    # 目标列名，S或MS任务中的目标特征
    target = 'OT'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'

    '''预测任务'''
    # 输入序列长度,这是用于模型训练的输入序列的长度
    seq_len = 60
    # 开始标记长度,这是模型输出目标中有标签数据的长度，类似于滑动窗口的长度
    label_len = 20
    # 预测序列长度
    pred_len = 1
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'
    inverse = False    # 反转输出数据

    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 38
    # decoder 输入特征数
    dec_in = 38
    # 输出通道数
    c_out = 38
    # 线性层隐含神经元个数
    d_model = 32
    # FFN 层隐含神经元个数
    d_ff = 32
    # 多头注意力机制
    n_heads = 8
    # encoder 层数
    e_layers = 2
    # decoder 层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 20
    # 对 Q 进行采样，对 Q 采样的因子数
    factor = 3
    # 是否下采样操作 pooling
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention
    output_attention = False

    '''优化'''
    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 500
    # batch size 大小
    batch_size = 60
    # early stopping 机制容忍次数
    patience = 5
    # 学习率
    learning_rate = 0.0001
    # 实验描述
    des = 'stock'
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = True
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2


# 创建参数对象
args = Args()

In [18]:
stock_down = ak.stock_cy_a_spot_em() # 创业板实时数据
stock_list = stock_down[~stock_down['名称'].str.contains("退|ST") & (stock_down['流通市值'] <= 1e11) & (stock_down['总市值'] >= 45e8)] # 去除退市和ST股票
# 保存数据，编码格式为utf-8
file_name = 'Stock_list.csv'
stock_list.to_csv(args.root_path + file_name,index=False,encoding='utf-8-sig')

In [20]:
from sklearn.preprocessing import StandardScaler

all_data_raw = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储原始数据
all_data_scaled = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储标准化后的数据
processed_count = 0  # 初始化计数器
total_count = len(stock_list["代码"].values)  # 获取总股票数量
scaler = StandardScaler()

for i in stock_list["代码"].values:
    NUM = i
    try:
        # 下载原始数据
        raw_data = download_data(NUM, args)
        # 更新已处理股票数量计数器
        processed_count += 1
        # 检查数据长度，如果小于300则跳过此次循环
        if raw_data.shape[0] < 300:
            print(f"股票代码 {NUM} 的数据长度小于300,跳过此次循环。")
            continue
        
        # 拼接数据，添加各种参数
        ad_data = add_data(raw_data.copy(), args)
        # 添加预测标签
        ot_data = add_label(ad_data.copy(), args)
        # 删除无效数据
        su_data = sub_data(ot_data.copy(), args)

        # 存储未经标准化的数据
        all_data_raw = pd.concat([all_data_raw, su_data], ignore_index=True)

        # 标准化处理
        # scaler = StandardScaler()
        non_time_columns = su_data.columns[1:-1]  # 假设时间列是第一列
        su_data_scaled = su_data.copy()
        su_data_scaled[non_time_columns] = scaler.fit_transform(su_data[non_time_columns])

        # 存储经过标准化的数据
        all_data_scaled = pd.concat([all_data_scaled, su_data_scaled], ignore_index=True)

        # 如果args.data_addzero为True，并且all_data不是空的，先插入20行全为0的数据
        if args.data_addzero and not all_data_raw.empty:
            zero_data_raw = pd.DataFrame(0, index=range(args.data_addzero), columns=all_data_raw.columns)
            all_data_raw = pd.concat([all_data_raw, zero_data_raw], ignore_index=True)

        # 如果args.data_addzero为True，并且all_data_scaled不是空的，先插入20行全为0的数据
        if args.data_addzero and not all_data_scaled.empty:
            zero_data_scaled = pd.DataFrame(0, index=range(args.data_addzero), columns=all_data_scaled.columns)
            all_data_scaled = pd.concat([all_data_scaled, zero_data_scaled], ignore_index=True)

    except Exception as e:
        print(f"处理股票代码 {NUM} 时出现错误: {e}")
        continue

    # 计算并打印处理进度
    progress = processed_count / total_count
    print(f"处理进度: {processed_count}/{total_count} ({progress:.2%})")
    print(f"当前all_data的形状: {all_data_scaled.shape}")

# 保存原始数据到 CSV 文件
file_name_raw = "all_data_raw.csv"
all_data_raw.to_csv(args.root_path + file_name_raw, index=False)

# 保存标准化后的数据到 CSV 文件
file_name_scaled = "all_data_scaled.csv"
all_data_scaled.to_csv(args.root_path + file_name_scaled, index=False)
print("数据保存完毕。")

获取数据时间为： 20151201 - 20231220
原始数据形状： (493, 9)
添加数据以后形状： (493, 35)
添加label以后数据形状: (493, 37)
删除指定行、列后数据形状:  (463, 35)
处理进度: 1/608 (0.16%)
当前all_data的形状: (523, 35)
获取数据时间为： 20151201 - 20231220
原始数据形状： (1796, 9)
添加数据以后形状： (1796, 35)
添加label以后数据形状: (1796, 37)
删除指定行、列后数据形状:  (1766, 35)
处理进度: 2/608 (0.33%)
当前all_data的形状: (2349, 35)
获取数据时间为： 20151201 - 20231220
原始数据形状： (1924, 9)
添加数据以后形状： (1924, 35)
添加label以后数据形状: (1924, 37)
删除指定行、列后数据形状:  (1894, 35)
处理进度: 3/608 (0.49%)
当前all_data的形状: (4303, 35)
获取数据时间为： 20151201 - 20231220
原始数据形状： (1183, 9)
添加数据以后形状： (1183, 35)
添加label以后数据形状: (1183, 37)
删除指定行、列后数据形状:  (1153, 35)
处理进度: 4/608 (0.66%)
当前all_data的形状: (5516, 35)
获取数据时间为： 20151201 - 20231220
原始数据形状： (1158, 9)
添加数据以后形状： (1158, 35)
添加label以后数据形状: (1158, 37)
删除指定行、列后数据形状:  (1128, 35)
处理进度: 5/608 (0.82%)
当前all_data的形状: (6704, 35)
获取数据时间为： 20151201 - 20231220
原始数据形状： (1956, 9)
添加数据以后形状： (1956, 35)
添加label以后数据形状: (1956, 37)
删除指定行、列后数据形状:  (1926, 35)
处理进度: 6/608 (0.99%)
当前all_data的形状: (8690, 35)
获取数据时间为： 

In [5]:
# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


# 获取列数
num_columns = all_data_raw.shape[1]
# args.des = NUM
args.enc_in = num_columns - 1
args.dec_in = num_columns - 1
args.c_out = num_columns - 1

In [ ]:
# 检查并设置 GPU
args.use_gpu = torch.cuda.is_available() and args.use_gpu
if args.use_gpu:
    print("使用 GPU.")
    total_cuda_devices = torch.cuda.device_count()  # 获取系统中可用的 GPU 总数
    print(f"系统中总共有 {total_cuda_devices} 个 CUDA 设备可用。")
    if args.use_multi_gpu:
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]
        
        # 打印多 GPU 使用情况
        print(f"使用多个GPU: {args.device_ids}")
        device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
        print(f"Primary GPU (cuda:{args.gpu}) is in use.")
    else:
        args.gpu = 0
        device = torch.device("cuda" if args.use_gpu else "cpu")
        print("使用单个 GPU.")
else:
    device = torch.device("cpu")
    print("使用 CPU.")

# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
elif args.task_name == 'short_term_forecast':
    Exp = Exp_Short_Term_Forecast
elif args.task_name == 'imputation':
    Exp = Exp_Imputation
elif args.task_name == 'anomaly_detection':
    Exp = Exp_Anomaly_Detection
elif args.task_name == 'classification':
    Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况

# 进行训练和测试
if args.is_training:
    for ii in range(args.itr):
        # 实验记录设置
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # 设置实验
        print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        if args.use_gpu:
            torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # 设置实验
    print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    if args.use_gpu:
        torch.cuda.empty_cache()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/